# Quick Start

EvoX is a distributed GPU-accelerated framework for scalable evolutionary computation.

In [2]:
# install evox, skip it if you have already installed evox
try:
    import evox
except ImportError:
    !pip install --disable-pip-version-check --upgrade -q evox
    import evox

In [3]:
from evox import algorithms, problems, workflows, monitors

import jax.numpy as jnp
from jax import random

## Create an algorithm and a problem

To demostrate, we will create a PSO algorithm and an Ackley function.

For more detailed list, please refer to our API documentation. [List of Algorithms](https://evox.readthedocs.io/en/latest/api/algorithms/index.html) and [List of Problems](https://evox.readthedocs.io/en/latest/api/problems/index.html).

In [4]:
pso = algorithms.PSO(
    lb=jnp.full(shape=(2,), fill_value=-32),
    ub=jnp.full(shape=(2,), fill_value=32),
    pop_size=100,
)
ackley = problems.numerical.Ackley()

Now we want to run the algorithm against the problem.
To accomplish this, we need to create a `workflow` which represents the overall process of evolutionary computation.

In [5]:
workflow = workflows.StdWorkflow(pso, ackley)

Since we adopt the functional programming paradigm.
We must explicitly initialize and use the state of a module.
To initialize, call `init` with a pseudorandom number generators key (PRNGKey).

In [6]:
key = random.PRNGKey(42)
state = workflow.init(key)

The `state` represents the mutatable variables within the whole workflow, including those inside the algorithm and the problem. For example, the `population` in an algorithm is part of the `state`, because it will be changing across iterations.

Now, call `step` on the workflow to execute one iteration.

In [7]:
state = workflow.step(state)

To run multiple iterations, wrap it inside a for-loop.

In [8]:
# run the workflow for 100 steps
for i in range(100):
    state = workflow.step(state)

Notice that we are passing `state` as an argument of `step` and it returns a new `state`. This is exactly how the stateful computation works in functional programming.

And you may also notice that the `step` doesn't give any feedback, like the result of the optimization. This is because we are missing another component in our workflow.
Introducing `monitor`.

## Monitor

Monitor is a standard way to monitor the intermediate values inside a optimization process. Information like fitness or population can be easily obtained by the monitor.

Now, create a "Standard single-objective monitor"

In [9]:
monitor = monitors.StdSOMonitor()

The monitor can be plugged into the workflow.

In [10]:
workflow = workflows.StdWorkflow(
    pso,
    ackley,
    monitors=[monitor],
)

Now, re-initialize the workflow, and executed it again.

In [11]:
# init the workflow
state = workflow.init(key)
# run the workflow for 5 steps
for i in range(5):
    state = workflow.step(state)

This time, we can access the minimum fitness achieved through the `monitor`.

In [12]:
# since monitor works asynchronously
# flush make sure all the data is collected
monitor.flush()
monitor.get_best_fitness()

Array(4.297701, dtype=float32)

Execute another 5 iterations, and the minimum fitness will change accordingly.

In [13]:
for i in range(5):
    state = workflow.step(state)

# since we run another 5 iterations, the result will get better.
monitor.flush()
monitor.get_best_fitness()

Array(1.2975216, dtype=float32)

To record information related to the population as well, turn on `record_pop` option in the workflow.

This will cause the population be sent to the monitor as well, and the monitor can then determine the best solution on the fly.

Please note that when the population size is very large, is option can harm the overall performance.

In [14]:
# create a new monitor and workflow
monitor = monitors.StdSOMonitor()
workflow = workflows.StdWorkflow(
    pso,
    ackley,
    monitors=[monitor],
    record_pop = True, # <- use this!
)

state = workflow.init(key)
for i in range(10):
    state = workflow.step(state)

In [15]:
monitor.flush()
monitor.get_best_fitness()

Array(1.2975216, dtype=float32)

In [16]:
monitor.get_best_solution()

Array([-0.02271652, -0.19201761], dtype=float32)

It show the best solution is (-0.02271652, -0.19201761), which is close to the global minimum at (0,0). 🥳

Additionally, please note that the best fitness remains the same as last time. This is because we are using the same `key` when initializing the workflow as before. This deterministic behavior in EvoX allows others to easily reproduce your results.